Uji ke 10 Data awal

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# Paths & params
TRAIN_CSV   = '/kaggle/input/data-btc/fix3/fix2/train.csv'
VAL_CSV     = '/kaggle/input/data-btc/fix3/fix2/val.csv'
TEST_CSV    = '/kaggle/input/data-btc/fix3/fix2/test.csv'
WINDOW_SIZE = 48
BATCH_SIZE  = 64
TUNE_EPOCHS = 5
FINAL_EPOCHS= 5
LR          = 1e-3
PATIENCE    = 3
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED        = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

# 1) Load raw
df_tr_raw   = pd.read_csv(TRAIN_CSV, parse_dates=['timestamp'])
df_va_raw   = pd.read_csv(VAL_CSV,   parse_dates=['timestamp'])
df_te_raw   = pd.read_csv(TEST_CSV,  parse_dates=['timestamp'])

# 2) Define horizons
HORIZONS = ['1h','2h','3h','6h','12h','1d','3d','7d','15d','30d']
LABELS   = [f'label_{h}' for h in HORIZONS]
FEATS    = [c for c in df_tr_raw.columns if c not in LABELS+['timestamp','close_time']]

# 3) Scale train & val
scaler_X = MinMaxScaler().fit(df_tr_raw[FEATS])
scaler_y = MinMaxScaler().fit(df_tr_raw[LABELS])

df_tr = df_tr_raw.copy()
df_va = df_va_raw.copy()
df_tr[FEATS] = scaler_X.transform(df_tr[FEATS])
df_va[FEATS] = scaler_X.transform(df_va[FEATS])
df_tr[LABELS] = scaler_y.transform(df_tr[LABELS])
df_va[LABELS] = scaler_y.transform(df_va[LABELS])

# 4) make_sequences
def make_sequences(df_scaled, raw_close, raw_label, window):
    X, Y, LP, Y_true = [], [], [], []
    arr_f = df_scaled[FEATS].values
    N     = len(df_scaled)
    for i in range(N - window):
        seq        = arr_f[i:i+window]
        last_price = raw_close[i+window-1]
        future     = raw_label[i+window]
        delta      = (future - last_price) / (last_price + 1e-8)
        X.append(seq); Y.append(delta)
        LP.append(last_price); Y_true.append(future)
    if not X:
        return None,None,None,None
    return np.stack(X), np.stack(Y), np.array(LP), np.stack(Y_true)

# 5) Build train+val sequences
raw_close_tv = pd.concat([df_tr_raw, df_va_raw], ignore_index=True)['close'].values
raw_label_tv = pd.concat([df_tr_raw[LABELS], df_va_raw[LABELS]], ignore_index=True).values
df_tv_scaled = pd.concat([df_tr, df_va], ignore_index=True)

X_all, Y_all, LP_all, TRUE_all = make_sequences(
    df_tv_scaled, raw_close_tv, raw_label_tv, WINDOW_SIZE
)
n_tr_seq = len(df_tr_raw) - WINDOW_SIZE
X_tr, Y_tr = X_all[:n_tr_seq], Y_all[:n_tr_seq]
LP_tr, TRUE_tr = LP_all[:n_tr_seq], TRUE_all[:n_tr_seq]
X_va, Y_va = X_all[n_tr_seq:], Y_all[n_tr_seq:]
LP_va, TRUE_va = LP_all[n_tr_seq:], TRUE_all[n_tr_seq:]

# 6) Build TEST sequences (prepend last WINDOW_SIZE of val)
raw_close_te = np.concatenate([
    df_va_raw['close'].values[-WINDOW_SIZE:],
    df_te_raw['close'].values
])
raw_label_te = np.concatenate([
    df_va_raw[LABELS].values[-WINDOW_SIZE:],
    df_te_raw[LABELS].values
])
df_va_scaled       = df_va.copy()
df_va_scaled[FEATS]= scaler_X.transform(df_va_scaled[FEATS])
df_te_scaled       = df_te_raw.copy()
df_te_scaled[FEATS]= scaler_X.transform(df_te_scaled[FEATS])
df_comb_scaled     = pd.concat([
    df_va_scaled.iloc[-WINDOW_SIZE:],
    df_te_scaled[FEATS]
], ignore_index=True)

X_te, Y_te, LP_te, TRUE_te = make_sequences(
    df_comb_scaled, raw_close_te, raw_label_te, WINDOW_SIZE
)

# 7) DataLoaders
train_ds = TensorDataset(torch.FloatTensor(X_tr), torch.FloatTensor(Y_tr))
val_ds   = TensorDataset(torch.FloatTensor(X_va), torch.FloatTensor(Y_va))
test_ds  = None if X_te is None else TensorDataset(torch.FloatTensor(X_te), torch.FloatTensor(Y_te))

train_ld = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_ld   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)
test_ld  = None if test_ds is None else DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# 8) Model (unchanged)
class MultiHorizonModel(nn.Module):
    def __init__(self, n_feat, n_hor, d_emb=8):
        super().__init__()
        self.conv = nn.Conv1d(n_feat, 64, kernel_size=3, padding=2, dilation=2)
        self.lstm = nn.LSTM(64, 128, batch_first=True, bidirectional=True)
        self.norm = nn.LayerNorm(256)
        self.attn = nn.MultiheadAttention(256, num_heads=4, batch_first=True)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.h_emb= nn.Embedding(n_hor, d_emb)
        self.heads= nn.ModuleList([
            nn.Sequential(nn.Linear(256+d_emb,64), nn.ReLU(),
                          nn.Dropout(0.2), nn.Linear(64,1))
            for _ in range(n_hor)
        ])
    def forward(self,x):
        b = x.size(0)
        x = x.permute(0,2,1)
        x = self.conv(x)
        x = x.permute(0,2,1)
        x,_ = self.lstm(x)
        x = self.norm(x)
        att,_ = self.attn(x,x,x)
        x = self.norm(x+att)
        x = x.permute(0,2,1)
        feat = self.pool(x).squeeze(-1)
        outs=[]
        for i, head in enumerate(self.heads):
            emb = self.h_emb(torch.full((b,), i, dtype=torch.long, device=feat.device))
            outs.append(head(torch.cat([feat,emb],1)))
        return torch.cat(outs,1)

# eval helper
def eval_phase(model, LP, TRUE, desc):
    print(f"--- {desc} LP first10:", LP[:10])
    for hi,h in enumerate(HORIZONS):
        print(f"--- {desc} TRUE_{h} first10:", TRUE[:10,hi])
    preds=[]; trues=TRUE[:10]
    model.eval()
    with torch.no_grad():
        for i in range(10):
            xb = torch.FloatTensor(X_te[i:i+1]).to(DEVICE)
            out= model(xb).cpu().numpy()[0]
            preds.append(out)
    preds = np.stack(preds)  # (10,10)
    price_pred = np.zeros_like(preds)
    for i in range(10):
        price_pred[i] = LP[i] * (1+preds[i])
    # plot
    for hi,h in enumerate(HORIZONS):
        rmse = mean_squared_error(trues[:,hi], price_pred[:,hi], squared=False)
        mape = mean_absolute_percentage_error(trues[:,hi], price_pred[:,hi])*100
        print(f"{desc} {h}: RMSE={rmse:.2f}, MAPE={mape:.2f}%")
        plt.figure(figsize=(9,4))
        plt.plot(trues[:,hi], label='Actual')
        plt.plot(price_pred[:,hi],'--', label='Predicted')
        plt.xticks(range(10))
        plt.title(f"{desc} - {h}")
        plt.legend(); plt.tight_layout(); plt.show()

# 9) Phase 1: TRAIN→VAL (no tune)
print("=== PHASE 1: TRAIN→VAL ===")
model = MultiHorizonModel(n_feat=X_tr.shape[2], n_hor=len(HORIZONS)).to(DEVICE)
opt   = torch.optim.Adam(model.parameters(), lr=LR)
crit  = nn.HuberLoss(delta=1.0)
weights = torch.linspace(1,2,steps=len(HORIZONS)).to(DEVICE)

best_val, cnt = float('inf'), 0
for ep in range(1, TUNE_EPOCHS+1):
    model.train()
    for xb,yb in train_ld:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        loss  = (crit(model(xb), yb)*weights).mean()
        opt.zero_grad(); loss.backward(); opt.step()
    # validate
    val_losses=[]
    model.eval()
    with torch.no_grad():
        for xb,yb in val_ld:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            val_losses.append(((crit(model(xb), yb)*weights).mean().item()))
    mv = np.mean(val_losses)
    print(f"Ep{ep} val_loss={mv:.4f}")
    if mv<best_val:
        best_val, cnt = mv, 0
        torch.save(model.state_dict(), 'best_tune.pth')
    else:
        cnt+=1
        if cnt>=PATIENCE:
            print("Early stopping"); break

# eval on VAL sequences
eval_phase(model, LP_va, TRUE_va, desc="Phase1 VAL")

# 10) Phase 2: TRAIN+VAL→TEST (final retrain & eval)
print("\n=== PHASE 2: TRAIN+VAL→TEST ===")
model.load_state_dict(torch.load('best_tune.pth'))
opt = torch.optim.Adam(model.parameters(), lr=LR)
# combine train+val
X_comb = np.vstack([X_tr, X_va]); Y_comb = np.vstack([Y_tr, Y_va])
comb_ds  = TensorDataset(torch.FloatTensor(X_comb), torch.FloatTensor(Y_comb))
comb_ld  = DataLoader(comb_ds, batch_size=BATCH_SIZE, shuffle=True)
for ep in range(1, FINAL_EPOCHS+1):
    model.train()
    for xb,yb in comb_ld:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        loss  = (crit(model(xb), yb)*weights).mean()
        opt.zero_grad(); loss.backward(); opt.step()

# eval on TEST sequences
eval_phase(model, LP_te, TRUE_te, desc="Phase2 TEST")


Uji ke 100 data awal dan simpan model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os

# Paths & params
TRAIN_CSV    = '/kaggle/input/data-btc/fix3/fix2/train.csv'
VAL_CSV      = '/kaggle/input/data-btc/fix3/fix2/val.csv'
TEST_CSV     = '/kaggle/input/data-btc/fix3/fix2/test.csv'
WINDOW_SIZE  = 48
BATCH_SIZE   = 64
TUNE_EPOCHS  = 10
FINAL_EPOCHS = 10
LR           = 1e-3
PATIENCE     = 3
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED         = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

# 1) Load raw
df_tr_raw = pd.read_csv(TRAIN_CSV, parse_dates=['timestamp'])
df_va_raw = pd.read_csv(VAL_CSV,   parse_dates=['timestamp'])
df_te_raw = pd.read_csv(TEST_CSV,  parse_dates=['timestamp'])

# 2) Horizons & features
HORIZONS = ['1h','2h','3h','6h','12h','1d','3d','7d','15d','30d']
LABELS   = [f'label_{h}' for h in HORIZONS]
FEATS    = [c for c in df_tr_raw.columns if c not in LABELS+['timestamp','close_time']]

# 3) Scale train & val
scaler_X = MinMaxScaler().fit(df_tr_raw[FEATS])
scaler_y = MinMaxScaler().fit(df_tr_raw[LABELS])

df_tr = df_tr_raw.copy()
df_va = df_va_raw.copy()
df_tr[FEATS]  = scaler_X.transform(df_tr[FEATS])
df_va[FEATS]  = scaler_X.transform(df_va[FEATS])
df_tr[LABELS] = scaler_y.transform(df_tr[LABELS])
df_va[LABELS] = scaler_y.transform(df_va[LABELS])

# 4) Sequence builder
def make_sequences(df_scaled, raw_close, raw_label, window):
    X, Y, LP, Y_true = [], [], [], []
    arr_f = df_scaled[FEATS].values
    N     = len(df_scaled)
    for i in range(N - window):
        seq        = arr_f[i:i+window]
        last_price = raw_close[i+window-1]
        future     = raw_label[i+window]
        delta      = (future - last_price) / (last_price + 1e-8)
        X.append(seq); Y.append(delta)
        LP.append(last_price); Y_true.append(future)
    if not X:
        return None,None,None,None
    return np.stack(X), np.stack(Y), np.array(LP), np.stack(Y_true)

# 5) Build train+val sequences
raw_close_tv = pd.concat([df_tr_raw, df_va_raw], ignore_index=True)['close'].values
raw_label_tv = pd.concat([df_tr_raw[LABELS], df_va_raw[LABELS]], ignore_index=True).values
df_tv_scaled = pd.concat([df_tr, df_va], ignore_index=True)

X_all, Y_all, LP_all, TRUE_all = make_sequences(
    df_tv_scaled, raw_close_tv, raw_label_tv, WINDOW_SIZE
)
n_tr_seq = len(df_tr_raw) - WINDOW_SIZE
X_tr, Y_tr       = X_all[:n_tr_seq], Y_all[:n_tr_seq]
LP_tr, TRUE_tr   = LP_all[:n_tr_seq], TRUE_all[:n_tr_seq]
X_va, Y_va       = X_all[n_tr_seq:], Y_all[n_tr_seq:]
LP_va, TRUE_va   = LP_all[n_tr_seq:], TRUE_all[n_tr_seq:]

# 6) Build TEST sequences (prepend last WINDOW_SIZE of val)
raw_close_te = np.concatenate([df_va_raw['close'].values[-WINDOW_SIZE:], df_te_raw['close'].values])
raw_label_te = np.concatenate([df_va_raw[LABELS].values[-WINDOW_SIZE:], df_te_raw[LABELS].values])

df_va_scaled        = df_va.copy();   df_va_scaled[FEATS] = scaler_X.transform(df_va[FEATS])
df_te_scaled        = df_te_raw.copy(); df_te_scaled[FEATS] = scaler_X.transform(df_te_raw[FEATS])
df_comb_scaled      = pd.concat([df_va_scaled.iloc[-WINDOW_SIZE:], df_te_scaled[FEATS]], ignore_index=True)

X_te, Y_te, LP_te, TRUE_te = make_sequences(df_comb_scaled, raw_close_te, raw_label_te, WINDOW_SIZE)

# 7) DataLoaders
train_ld = DataLoader(TensorDataset(torch.FloatTensor(X_tr), torch.FloatTensor(Y_tr)),
                      batch_size=BATCH_SIZE, shuffle=True)
val_ld   = DataLoader(TensorDataset(torch.FloatTensor(X_va), torch.FloatTensor(Y_va)),
                      batch_size=BATCH_SIZE, shuffle=False)
test_ld  = None if X_te is None else DataLoader(TensorDataset(torch.FloatTensor(X_te), torch.FloatTensor(Y_te)),
                                                batch_size=BATCH_SIZE, shuffle=False)

# 8) Model
class MultiHorizonModel(nn.Module):
    def __init__(self, n_feat, n_hor, d_emb=8):
        super().__init__()
        self.conv = nn.Conv1d(n_feat,64,3,padding=2,dilation=2)
        self.lstm = nn.LSTM(64,128,batch_first=True,bidirectional=True)
        self.norm = nn.LayerNorm(256)
        self.attn = nn.MultiheadAttention(256, num_heads=4, batch_first=True)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.h_emb= nn.Embedding(n_hor, d_emb)
        self.heads= nn.ModuleList([
            nn.Sequential(nn.Linear(256+d_emb,64), nn.ReLU(),
                          nn.Dropout(0.2), nn.Linear(64,1))
            for _ in range(n_hor)
        ])
    def forward(self, x):
        b = x.size(0)
        x = x.permute(0,2,1)
        x = self.conv(x)
        x = x.permute(0,2,1)
        x,_ = self.lstm(x)
        x = self.norm(x)
        att,_ = self.attn(x,x,x)
        x = self.norm(x+att)
        x = x.permute(0,2,1)
        feat = self.pool(x).squeeze(-1)
        outs = []
        for i, head in enumerate(self.heads):
            emb = self.h_emb(torch.full((b,), i, dtype=torch.long, device=feat.device))
            outs.append(head(torch.cat([feat, emb],1)))
        return torch.cat(outs,1)

# 9) Eval helper
def eval_phase(model, LP, TRUE, desc, first_n=100):
    n_pts = min(first_n, LP.shape[0])
    print(f"--- {desc} LP first {n_pts}: {LP[:n_pts]}")
    for hi,h in enumerate(HORIZONS):
        print(f"--- {desc} TRUE_{h} first {n_pts}: {TRUE[:n_pts,hi]}")
    model.eval()
    preds = []
    with torch.no_grad():
        for i in range(n_pts):
            xb = torch.FloatTensor(X_te[i:i+1]).to(DEVICE)
            out= model(xb).cpu().numpy()[0]
            preds.append(out)
    preds = np.stack(preds)
    price_pred = np.zeros_like(preds)
    for i in range(n_pts):
        price_pred[i] = LP[i] * (1 + preds[i])
    price_true = TRUE[:n_pts]
    for hi,h in enumerate(HORIZONS):
        rmse = mean_squared_error(price_true[:,hi], price_pred[:,hi], squared=False)
        mape = mean_absolute_percentage_error(price_true[:,hi], price_pred[:,hi])*100
        print(f"{desc} {h}: RMSE={rmse:.2f}, MAPE={mape:.2f}%")
        plt.figure(figsize=(9,4))
        plt.plot(price_true[:,hi], label='Actual')
        plt.plot(price_pred[:,hi],'--', label='Predicted')
        plt.xticks(range(0, n_pts, max(1, n_pts//10)))
        plt.title(f"{desc} - {h}")
        plt.legend(); plt.tight_layout(); plt.show()

# 10) Phase 1: TRAIN→VAL (no tuning)
print("=== PHASE 1: TRAIN→VAL ===")
model = MultiHorizonModel(n_feat=X_tr.shape[2], n_hor=len(HORIZONS)).to(DEVICE)
opt   = torch.optim.Adam(model.parameters(), lr=LR)
crit  = nn.HuberLoss(delta=1.0)
weights = torch.linspace(1,2,steps=len(HORIZONS)).to(DEVICE)

best_val, cnt = float('inf'), 0
for ep in range(1, TUNE_EPOCHS+1):
    model.train()
    for xb,yb in train_ld:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        loss  = (crit(model(xb), yb)*weights).mean()
        opt.zero_grad(); loss.backward(); opt.step()
    # validate
    val_losses = []
    model.eval()
    with torch.no_grad():
        for xb,yb in val_ld:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            val_losses.append(((crit(model(xb), yb)*weights).mean().item()))
    mv = np.mean(val_losses)
    print(f"Ep{ep} val_loss={mv:.4f}")
    if mv < best_val:
        best_val, cnt = mv, 0
        torch.save(model.state_dict(), 'best_tune.pth')
    else:
        cnt += 1
        if cnt >= PATIENCE:
            print("Early stopping"); break

eval_phase(model, LP_va, TRUE_va, desc="Phase1 VAL", first_n=100)

# 11) Phase 2: TRAIN+VAL→TEST (final retrain, save & eval)
print("\n=== PHASE 2: TRAIN+VAL→TEST ===")
model.load_state_dict(torch.load('best_tune.pth'))
opt = torch.optim.Adam(model.parameters(), lr=LR)
X_comb = np.vstack([X_tr, X_va])
Y_comb = np.vstack([Y_tr, Y_va])
comb_ds = TensorDataset(torch.FloatTensor(X_comb), torch.FloatTensor(Y_comb))
comb_ld = DataLoader(comb_ds, batch_size=BATCH_SIZE, shuffle=True)

for ep in range(1, FINAL_EPOCHS+1):
    model.train()
    for xb,yb in comb_ld:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        loss  = (crit(model(xb), yb)*weights).mean()
        opt.zero_grad(); loss.backward(); opt.step()

# save final model once
save_path = '/kaggle/working/lstm_final.pth'
torch.save(model.state_dict(), save_path)
print(f"Saved final LSTM model to {save_path}")

# evaluate
if test_ld is not None:
    eval_phase(model, LP_te, TRUE_te, desc="Phase2 TEST", first_n=100)
else:
    print("Test split too small, skipping.")
